In [3]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.0-bin-hadoop2.7/')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("Log_reg").getOrCreate()

In [7]:
df = spark.read.csv('titanic.csv', inferSchema = True, header = True)

In [8]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorIndexer, VectorAssembler

In [17]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='sex_vec')

In [18]:
gender_vec = OneHotEncoder(inputCol='sex_vec', outputCol ='sex_enc')

In [19]:
embark_indexer = StringIndexer(inputCol = "Embarked", outputCol="Embarked_vec")

In [20]:
embark_vec = OneHotEncoder( inputCol="Embarked_vec", outputCol="embark_enc")

In [21]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [22]:
myCol = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [23]:
myCol.show()  #A separate column is created for simplicity and ease of computation

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     3|  male|null|    0|    0| 8.4583|       Q|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       

In [51]:
final_data = myCol.na.drop()               #To drop columns with Null values

In [65]:
assembler = VectorAssembler(inputCols=['Pclass', 'sex_enc', 'Age', 'SibSp', 'Parch', 'Fare', 'embark_enc'], 
                           outputCol='features')

In [66]:
from pyspark.ml.classification import LogisticRegression

In [67]:
log_reg_model = LogisticRegression(featuresCol="features", labelCol="Survived")

In [68]:
from pyspark.ml import Pipeline

In [69]:
pipeline = Pipeline(stages = [gender_indexer,gender_vec,embark_indexer, embark_vec, assembler, log_reg_model])

In [70]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [71]:
tr_data = train_data.summary

In [72]:
train_data.summary

<bound method DataFrame.summary of DataFrame[Survived: int, Pclass: int, Sex: string, Age: double, SibSp: int, Parch: int, Fare: double, Embarked: string]>

In [73]:
fit_model = pipeline.fit(train_data)

In [74]:
transform_model = fit_model.transform(test_data)

In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [78]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Survived")

In [79]:
transform_model.select('prediction', 'Survived').show()

+----------+--------+
|prediction|Survived|
+----------+--------+
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
+----------+--------+
only showing top 20 rows



In [80]:
AUC = evaluator.evaluate(transform_model)

In [81]:
AUC

0.8169372294372295